In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import math
import datetime
from pandas_datareader import DataReader

import backtrader as bt

import backtrader.analyzers as btanalyzers
import talib

In [3]:
class Strategy(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.expo =  bt.ind.ExponentialMovingAverageOscillator(self.datas[0],period=20)
        self.atr = bt.ind.ATR(self.datas[0], plot = False)
        self.sma = bt.indicators.SimpleMovingAverage(
                      self.datas[0], period=20)
        self.sma1 = bt.indicators.SimpleMovingAverage(
                      self.datas[0], period=10)

        self.upper = self.expo + self.atr
        self.lower = self.expo - self.atr
        self.close = self.datas[0].close  


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: {:.2f}, Cost: {:.2f}, Comm {:.2f}'.format(
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: {:.2f}, Cost: {:.2f}, Comm {:.2f}'.format(
                    order.executed.price,
                    order.executed.value,
                    order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))


    def next(self):
        if not self.position:
            if self.close[0] > self.sma[0]:
                self.order = self.order_target_percent(target=0.9)
        else:
            if self.close[0]< self.sma1[0]:
                self.order = self.sell()

In [4]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    
    df = pd.read_csv("sh500.csv", encoding='gbk',skiprows=0) 
    df = df[['日期','收盘价','最高价','最低价','开盘价','成交量']]
    df.rename(columns={"日期":"datetime","开盘价":"open","最高价":"high","最低价'":"low","收盘价":"close","成交量":"volume"},inplace=True)
    df['datetime'] = pd.to_datetime(df['datetime'],format="%Y-%m-%d")
    df.set_index("datetime", inplace=True)
    df = df.sort_index(ascending=True)
    data = bt.feeds.PandasData(dataname=df)
#         dataname = dataname,
# #         fromdate = datetime.datetime(2015, 1, 1),
# #         todate = datetime.datetime(2020,12,14),
#         dtformat='%Y-%m-%d',
#         timeframe = bt.TimeFrame.Days,
#         datetime=0,
#         nullvalue=0,
#         open=6,
#         high=4,
#         low=5,
#         close=3,
#         volume=10
#     )
    
    cerebro.adddata(data)


    cerebro.addstrategy(Strategy)
    cerebro.broker.setcash(100000)
    cerebro.broker.setcommission(commission=0)
    cerebro.addsizer(bt.sizers.PercentSizer, percents=98)

    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name = 'sharpe')
    cerebro.addanalyzer(btanalyzers.DrawDown, _name = 'drawdown')
    cerebro.addanalyzer(btanalyzers.Returns, _name='returns')

    print('Starte Portfolio Value {}'.format(cerebro.broker.getvalue()))
    back = cerebro.run()
#     back = back[0]
#     back.
    print('end portfolio value {}'.format(cerebro.broker.getvalue()))
#     print('夏普比率:', back.analyzers.sharpe.get_analysis())
#     print('回撤指标:', back.analyzers.DW.get_analysis())
    par_list = [[x.analyzers.returns.get_analysis()['rtot'],
                 x.analyzers.returns.get_analysis()['rnorm100'],
                 x.analyzers.drawdown.get_analysis()['max']['drawdown'],
                 x.analyzers.sharpe.get_analysis()['sharperatio']
                 ] for x in back]
    par_df = pd.DataFrame(par_list, columns=['Total Return','APR', 'Drawdown', 'SharpRatio'])
    print(par_df)


    cerebro.plot(style='candle')

FileNotFoundError: [Errno 2] No such file or directory: 'sh500.csv'